In [30]:
%serialconnect to --port=/dev/ttyUSB0 --baud=115200

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


Keywords: micropython, hcsr04, sonar sensor, NodeMCU

In [31]:
import dht

In [32]:
import machine

In [33]:
d = dht.DHT11(machine.Pin(4))

In [51]:
d.measure()

In [52]:
d.temperature()

In [53]:
print(d.temperature())

25


In [56]:
# 77F = 25C

In [57]:
d.humidity()

In [58]:
print(d.humidity())

39


In [60]:
print(dir(d))

['__class__', '__init__', '__module__', '__qualname__', '__dict__', 'buf', 'pin', 'measure', 'humidity', 'temperature']


In [34]:
import machine
import utime

trigger_pin=16
echo_pin=5

# https://stackoverflow.com/questions/42416224/micropython-hc-sr04-esp8266
trig=machine.Pin(trigger_pin, machine.Pin.OUT)
while True:
    trig.off() # stop reading
    utime.sleep_us(2)
    trig.on()
    utime.sleep_us(10)
    trig.off()
    echo=machine.Pin(echo_pin, machine.Pin.IN)
    while echo.value() == 0:
        pass
    t1 = utime.ticks_us()
    while echo.value() == 1:
        pass
    t2 = utime.ticks_us()
    cm = (t2 - t1) / 58.0
    print(cm)
    utime.sleep(2)

15.4483
41.069
50.3965
.18.7586
16.8103
14.3103
.21.3793
23.7931
.24.0172
37.7414
47.6207
.67.1896


*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 24, in <module>
KeyboardInterrupt: 


In [43]:
from machine import Pin
import time
# https://github.com/rsc1975/micropython-hcsr04/blob/master/hcsr04.py
class HCSR04:
    """
    Driver to use the untrasonic sensor HC-SR04.
    The sensor range is between 2cm and 4m.
    The timeouts received listening to echo pin are converted to OSError('Out of range')
    """
    # echo_timeout_us is based in chip range limit (400cm)
    def __init__(self, trigger_pin, echo_pin, echo_timeout_us=500*2*30):
        """
        trigger_pin: Output pin to send pulses
        echo_pin: Readonly pin to measure the distance. The pin should be protected with 1k resistor
        echo_timeout_us: Timeout in microseconds to listen to echo pin. 
        By default is based in sensor limit range (4m)
        """
        self.echo_timeout_us = echo_timeout_us
        # Init trigger pin (out)
        self.trigger = Pin(trigger_pin, mode=Pin.OUT, pull=None)
        self.trigger.value(0)

        # Init echo pin (in)
        self.echo = Pin(echo_pin, mode=Pin.IN, pull=None)

    def _send_pulse_and_wait(self):
        """
        Send the pulse to trigger and listen on echo pin.
        We use the method `machine.time_pulse_us()` to get the microseconds until the echo is received.
        """
        self.trigger.value(0) # Stabilize the sensor
        time.sleep_us(5)
        self.trigger.value(1)
        # Send a 10us pulse.
        time.sleep_us(10)
        self.trigger.value(0)
        try:
            pulse_time = machine.time_pulse_us(self.echo, 1, self.echo_timeout_us)
            return pulse_time
        except OSError as ex:
            if ex.args[0] == 110: # 110 = ETIMEDOUT
                raise OSError('Out of range')
            raise ex

    def distance_mm(self):
        """
        Get the distance in milimeters without floating point operations.
        """
        pulse_time = self._send_pulse_and_wait()

        # To calculate the distance we get the pulse_time and divide it by 2 
        # (the pulse walk the distance twice) and by 29.1 becasue
        # the sound speed on air (343.2 m/s), that It's equivalent to
        # 0.34320 mm/us that is 1mm each 2.91us
        # pulse_time // 2 // 2.91 -> pulse_time // 5.82 -> pulse_time * 100 // 582 
        mm = pulse_time * 100 // 582
        return mm

    def distance_cm(self):
        """
        Get the distance in centimeters with floating point operations.
        It returns a float
        """
        pulse_time = self._send_pulse_and_wait()

        # To calculate the distance we get the pulse_time and divide it by 2 
        # (the pulse walk the distance twice) and by 29.1 becasue
        # the sound speed on air (343.2 m/s), that It's equivalent to
        # 0.034320 cm/us that is 1cm each 29.1us
        cms = (pulse_time / 2) / 29.1
        return cms

In [44]:
echo = HCSR04(trigger_pin, echo_pin)

In [48]:
print(echo.distance_cm())

53.8316


In [62]:
trig=machine.Pin(trigger_pin, machine.Pin.OUT)
echo=machine.Pin(echo_pin, machine.Pin.IN)
method2 = HCSR04(trigger_pin, echo_pin)
while True:
    trig.off() # stop reading
    utime.sleep_us(2)
    trig.on()
    utime.sleep_us(10)
    trig.off()
    while echo.value() == 0:
        pass
    t1 = utime.ticks_us()
    while echo.value() == 1:
        pass
    t2 = utime.ticks_us()
    cm = (t2 - t1) / 58.0
    print(cm)
    utime.sleep(0.5)
    print(method2.distance_cm())
    utime.sleep(0.5)

29.0345
54.8969
53.9138
50.0515
28.8965
28.7113
27.5172
24.433
19.431
17.3196
.16.7241
16.2371
15.431
19.8282
17.3103
16.2371
16.7931
16.2027
16.7241
25.5326
.52.1897
22.3711
54.3448
56.9588
55.3276
55.0
57.2931
56.9416
54.2759
57.165
.54.1379
56.2371
55.3965
57.1134
56.7759
56.8041
57.8276
53.299
55.3276
55.6186
.56.4655
51.6323
55.7759

**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)


**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)

